# Tutorial 0: Curate Raw CCLE RNA-seq Data

In this tutorial we will download publicly available raw RNA-sequencing data, in the form of fastqs, from the Cancer Cell Line Encyclopedia (CCLE) Sequence Read Archive (SRA) repository. 

This CCLE expression dataset is for use in the demo, and will serve as the basis for generating raw data Quilt packages, and input to Nextflow nf-core pipelines. If available, we encourage you to sub in your in-house datasets in place of the demo CCLE dataset for a more practical use case tailored to your company's platform. 

Although this demo will be centered around an RNA-seq expression dataset and the corresponding `nf-core/rna-seq` pipeline, the principles and methods underlying the demo are generalizable across -omics datasets. 

In [47]:
import pandas as pd
import numpy as np

In [48]:
pd.set_option('display.max_columns', None)

# 1. Curate CCLE metadata

First, we want to define a subset of the larger CCLE collection to use for this demo. To define the demo cohort, we used two pieces of publicly available CCLE sample-level metadata: 
1. [Sequence Read Archive (SRA) Bioproject PRJNA523380](https://www.ncbi.nlm.nih.gov/sra?term=PRJNA523380): sample, sequencing & run metadata
2. [cBioportal study](https://www.cbioportal.org/study/summary?id=ccle_broad_2019): detailed biological & clinical sample metadata

Both of these raw metadata files are available in the `demo_data/` directory of this repo.

## 1.1 SRA metadata


In [51]:
# load SRA CCLE metadata
# rows are CCLE samples, columns are metadata fields
sra = pd.read_csv("./demo_data/sample_metadata/downloads/CCLE_SraRunTable.txt")
sra.head()

,Run,Age,Assay Type,AssemblyName,AvgSpotLen,Bases,BIOMATERIAL_PROVIDER,BioProject,BioSample,BioSampleModel,Bytes,cell_line,Center Name,Consent,DATASTORE filetype,DATASTORE provider,DATASTORE region,disease,disease_stage,ETHNICITY,Experiment,Instrument,isolate,Library Name,LibraryLayout,LibrarySelection,LibrarySource,Organism,Platform,ReleaseDate,Sample Name,sample_type,sex,SRA Study,tissue,create_date,version
0,SRR8670667,81.0,WGS,GCA_000001405.13,202,128791662980,ATCC:T24,PRJNA523380,SAMN10987918,Human,68905947744,T24,BROAD INSTITUTE,public,"bam,run.zq,sra","gs,ncbi,s3","gs.US,ncbi.public,s3.us-east-1",carcinoma (transitional_cell_carcinoma),primary,Caucasian,SRX5466713,Illumina HiSeq 2000,cell line,WGS-T24_URINARY_TRACT,PAIRED,size fractionation,GENOMIC,Homo sapiens,ILLUMINA,2019-03-27T00:00:00Z,T24_URINARY_TRACT,cell culture,female,SRP186687,urinary_tract,2019-03-05T22:40:00Z,1.0
1,SRR8670668,81.0,WGS,GCA_000001405.13,202,102955983372,NIBRI,PRJNA523380,SAMN10988334,Human,46845960495,SW948,BROAD INSTITUTE,public,"bam,run.zq,sra","gs,ncbi,s3","gs.US,ncbi.public,s3.us-east-1",carcinoma (adenocarcinoma),primary,Caucasian,SRX5466712,Illumina HiSeq 2000,cell line,WGS-SW948_LARGE_INTESTINE,PAIRED,size fractionation,GENOMIC,Homo sapiens,ILLUMINA,2019-03-27T00:00:00Z,SW948_LARGE_INTESTINE,cell culture,female,SRP186687,large_intestine,2019-03-05T17:38:00Z,1.0
2,SRR8670669,53.0,WGS,GCA_000001405.13,202,117381684900,NaN,PRJNA523380,SAMN10987793,Human,60335385048,SW 900,BROAD INSTITUTE,public,"bam,run.zq,sra","gs,ncbi,s3","gs.US,ncbi.public,s3.us-east-1",carcinoma (squamous_cell_carcinoma),primary,Caucasian,SRX5466711,Illumina HiSeq 2000,cell line,WGS-SW900_LUNG,PAIRED,size fractionation,GENOMIC,Homo sapiens,ILLUMINA,2019-03-27T00:00:00Z,SW900_LUNG,cell culture,male,SRP186687,lung,2019-03-05T19:22:00Z,1.0
3,SRR8670670,59.0,WGS,GCA_000001405.13,202,136392701588,GNF,PRJNA523380,SAMN10988339,Human,76410262878,SW579,BROAD INSTITUTE,public,"bam,run.zq,sra","gs,ncbi,s3","gs.US,ncbi.public,s3.us-east-1",carcinoma (anaplastic_carcinoma),primary,Caucasian,SRX5466710,Illumina HiSeq 2000,cell line,WGS-SW579_THYROID,PAIRED,size fractionation,GENOMIC,Homo sapiens,ILLUMINA,2019-03-27T00:00:00Z,SW579_THYROID,cell culture,male,SRP186687,thyroid,2019-03-05T21:09:00Z,1.0
4,SRR8670671,NaN,WGS,GCA_000001405.13,202,111933931548,JCRB:TE-1,PRJNA523380,SAMN10988294,Human,55384403411,TE-1,BROAD INSTITUTE,public,"bam,run.zq,sra","gs,ncbi,s3","gs.US,ncbi.public,s3.us-east-1",carcinoma (squamous_cell_carcinoma),primary,Asian,SRX5466709,Illumina HiSeq 2000,cell line,WGS-TE1_OESOPHAGUS,PAIRED,size fractionation,GENOMIC,Homo sapiens,ILLUMINA,2019-03-27T00:00:00Z,TE1_OESOPHAGUS,cell culture,male,SRP186687,oesophagus,2019-03-05T18:52:00Z,1.0


In [52]:
# confirm all CCLE samples are part of the same BioProject -- yes!
sra["BioProject"].value_counts()

BioProject
PRJNA523380    4550
Name: count, dtype: int64

In [53]:
# each sample+assay is its own run
sra["Run"].value_counts().head()

Run
SRR8670667    1
SRR8633850    1
SRR8633856    1
SRR8633855    1
SRR8633854    1
Name: count, dtype: int64

In [54]:
# subset to samples profiled with RNA-seq only (n=1,019)
display(sra["Assay Type"].value_counts())
sra_rna = sra.loc[sra["Assay Type"] == "RNA-Seq", :]
sra_rna.shape

Assay Type
RNA-Seq             1019
Targeted-Capture     976
AMPLICON             972
Bisulfite-Seq        928
WGS                  329
WXS                  326
Name: count, dtype: int64

(1019, 37)

## 1.2 Merge cBioportal & SRA metadata together 

SRA metadata has very detailed sequencing & sample processing information. To compliment this, we will add detailed clinical & biological information from the corresponding CCLE cBioportal study.

In [55]:
cbio = pd.read_csv("./demo_data/sample_metadata/downloads/cbio_ccle_broad_2019_clinical_data.tsv",
                   sep = "\t"
                  )
cbio.head()

,Study ID,Patient ID,Sample ID,Age,Annotation Source,Cancer Type,Cancer Type Detailed,Cell Line Source,Characteristics,DepMap ID,Disease Ontology,Doubling Time (hrs),Doubling Time From Vendor,Ethnicity Category,Fraction Genome Altered,Freezing Medium,Genome Doublings,Geographic Distribution,Growth Medium,Histology,Hist Subtype 1,Hist Subtype 2,Hist Subtype 3,Life Stage,Lineage,Lineage Molecular Subtype,Lineage Subtype,Lineage Sub-subtype,Mutation Count,Mutation Rate,Name,Oncotree Code,Pathologist Annotation,Ploidy,Primary Tumor Site,Proteomics 10-Plex ID,Proteomics TMT label,Purity,Race Category,Number of Samples Per Patient,Sample Type,Sex,Site of Finding,Site Subtype 1,Site Subtype 2,Site Subtype 3,Subtype,Supplements,TMB (nonsynonymous),Tumor Type,Type Refined
0,ccle_broad_2019,127399_SOFT_TISSUE,127399_SOFT_TISSUE,NaN,NaN,Soft Tissue Sarcoma,Synovial Sarcoma,NaN,NaN,ACH-001270,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,soft_tissue,NaN,synovial_sarcoma,NaN,163.0,NaN,NaN,SYNS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,Synovial,NaN,5.533333,NaN,NaN
1,ccle_broad_2019,1321N1_CENTRAL_NERVOUS_SYSTEM,1321N1_CENTRAL_NERVOUS_SYSTEM,NaN,CCLE,Glioma,Astrocytoma,NaN,NaN,ACH-001000,Brain Cancer,NaN,NaN,Caucasian,0.5648,NaN,2.0,NaN,NaN,Glioma,astrocytoma,NS,NS,NaN,central_nervous_system,NaN,glioma,astrocytoma,NaN,NaN,1321N1,ASTR,NaN,3.45,Central_Nervous_System,NaN,NaN,0.98,NaN,1,Primary,NaN,NaN,brain,NS,NS,Astrocytoma,NaN,NaN,glioma,glioma
2,ccle_broad_2019,143B_BONE,143B_BONE,13.0,CCLE,Bone Cancer,Osteosarcoma,NaN,NaN,ACH-001001,NaN,NaN,NaN,Caucasian,0.5014,NaN,2.0,NaN,NaN,Osteosarcoma,NS,NS,NS,NaN,bone,NaN,osteosarcoma,NaN,NaN,NaN,143B,OS,NaN,3.61,Bone,NaN,NaN,0.98,Caucasian,1,Primary,Female,NaN,NS,NS,NS,Osteosarcoma,NaN,NaN,osteosarcoma,osteosarcoma
3,ccle_broad_2019,201T_LUNG,201T_LUNG,NaN,NaN,Non-Small Cell Lung Cancer,Lung Adenocarcinoma,NaN,NaN,ACH-002089,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,lung,NaN,NSCLC,NSCLC_adenocarcinoma,290.0,NaN,NaN,LUAD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,"Non-Small Cell Lung Cancer (NSCLC), Adenocarci...",NaN,10.100000,NaN,lung_NSC
4,ccle_broad_2019,22RV1_PROSTATE,22RV1_PROSTATE,NaN,CCLE,Prostate Cancer,Prostate Adenocarcinoma,ATCC,Adherent epithelial,ACH-000956,prostate_cancer,58.4,NaN,Caucasian,0.2002,5%DMSO,0.0,NaN,RPMI-1640+10% FBS,Carcinoma,NS,NS,NS,NaN,prostate,NaN,prostate_adenocarcinoma,NaN,2186.0,539.830868,22Rv1,PRAD,Prostate:Carcinoma,2.13,Prostate,24.0,126,0.99,NaN,1,Primary,Male,NaN,NS,NS,NS,Adenocarcinoma,NaN,73.700000,prostate,prostate


In [56]:
# confirm SRA Sample names are present in cBio metadata -- yes!
sra_rna["Sample Name"].isin(cbio["Sample ID"]).value_counts()

Sample Name
True    1019
Name: count, dtype: int64

In [57]:
# merge cBio & sra metadata 
cbio.set_index("Sample ID", inplace=True)
cbio = cbio.drop(columns="Age")
meta = sra_rna.merge(cbio, left_on = "Sample Name", right_index = True, how = "left")

# 2. Metadata cleanup

Now that we have a big table of comprehensive metadata, we want to harmonize inconsistencies & column header format, as well as drop repetitive columns with redundant information on the sample (mainly an artifact of pulling metadata fro two sources, there will be some overlap in information).

Having clean metadata is essential to making your data findable! Harmonizing spelling, formatting etc. upfront in the early stages of a project can be extremely powerful & time-saving to having clean covariates when it comes time for analysis. But more on that in Tutorial #1 where we cover metadata schemas & workflows for Quilt packages!

## 2.1 Harmonize freezing medium

Ugly mix of character types & naming inconsistencies decribing the same sample feature. 

For example, `5%DMSO` vs. `5% DMSO` vs. `5%DMSo` vs. `5` vs. `5 % DMSO`. 

The metadata lists the same 5% DMSO variable 5 different ways! Lets harmonize this...

In [58]:
# Oof, thats ugly
meta["Freezing Medium"].value_counts()

Freezing Medium
5%DMSO        242
5% DMSO       221
10%DMSO        45
10% DMSO        9
?               8
5%DMSo          2
5               1
5-7.5%DMSO      1
5 % DMSO        1
0.05            1
Name: count, dtype: int64

In [59]:
meta["Freezing Medium"] = meta["Freezing Medium"].str.replace("5%DMSO", "5% DMSO")
meta["Freezing Medium"] = meta["Freezing Medium"].str.replace("5%DMSo", "5% DMSO")
meta["Freezing Medium"] = meta["Freezing Medium"].str.replace("5 % DMSO", "5% DMSO")
meta["Freezing Medium"] = meta["Freezing Medium"].str.replace("10%DMSO", "10% DMSO")
meta["Freezing Medium"] = meta["Freezing Medium"].str.replace("5", "5% DMSO")
meta["Freezing Medium"] = meta["Freezing Medium"].str.replace("5% DMSO% DMSO", "5% DMSO")

In [60]:
# replace question mark with np.nan 
# to be consistent with unknown var used in other columns
meta["Freezing Medium"] = meta["Freezing Medium"].replace("?", np.nan)

In [61]:
# ah, looks much better :) 
meta["Freezing Medium"].value_counts()

Freezing Medium
5% DMSO              467
10% DMSO              54
5% DMSO-7.5% DMSO      1
0.05% DMSO             1
Name: count, dtype: int64

## 2.2 Drop redundant columns

In [62]:
# define columns to remove
# these columns contain the same information as another column
remove = ["Doubling Time From Vendor", 
          "Ethnicity Category",
          "Sex",
          "Hist Subtype 3",
          "Race Category",
          "Number of Samples Per Patient",
          "Sample Type",
          "Site Subtype 3",
          "Tumor Type",
          "Type Refined"
         ]
len(remove)

10

In [63]:
meta = meta.loc[:, ~meta.columns.isin(remove)]
meta.shape

(1019, 76)

## 2.3 Harmonize column formatting

Use CamelCase for column headers - make column headers with same format between SRA & cBioportal. This will make our lives easier when defining the metadata schema & workflow, and also...looks nicer!

In [64]:
col_update = {"Assay Type": "AssayType",
              "BIOMATERIAL_PROVIDER": "BiomaterialProvider",
              "cell_line": "CellLine",
              "Center Name": "CenterName",
              "DATASTORE filetype": "DataStoreFileType",
              "DATASTORE provider": "DataStoreProvider",
              "DATASTORE region": "DataStoreRegion",
              "disease": "Disease",
              "disease_stage": "DiseaseStage",
              "ETHNICITY": "Ethnicity",
              "isolate": "Isolate",
              "Library Name": "LibraryName",
              "Sample Name": "SampleName",
              "sample_type": "SampleType",
              "sex": "Sex",
              "SRA Study": "SRAStudy",
              "tissue": "Tissue",
              "create_date": "CreateDate",
              "create_date_batch": "CreateDateBatch",
              "version": "Version",
              "Study ID": "StudyID",
              "Patient ID": "PatientID",
              "Annotation Source": "AnnotationSource",
              "Cancer Type": "CancerType",
              "Cancer Type Detailed": "CancerTypeDetailed",
              "Cell Line Source": "CellLineSource",
              "DepMap ID": "DepMapID",
              "Disease Ontology": "DiseaseOntology",
              "Doubling Time (hrs)": "DoublingTimeHrs",
              "Fraction Genome Altered": "FractionGenomeAltered",
              "Freezing Medium": "FreezingMedium",
              "Genome Doublings": "GenomeDoublings",
              "Geographic Distribution": "GeographicDistribution",
              "Growth Medium": "GrowthMedium",
              "Hist Subtype 1": "HistologySubtype1",
              "Hist Subtype 2": "HistologySubtype2",
              "Life Stage": "LifeStage",
              "Lineage Molecular Subtype": "LineageMolecularSubtype",
              "Lineage Subtype": "LineageSubtype",
              "Lineage Sub-subtype": "LineageSubSubtype",
              "Mutation Count": "MutationCount",
              "Mutation Rate": "MutationRate",
              "Name": "CellLineNickName",
              "Oncotree Code": "OncotreeCode",
              "Pathologist Annotation": "PathologistAnnotation",
              "Primary Tumor Site": "PrimaryTumorSite",
              "Proteomics 10-Plex ID": "Proteomics10PlexID",
              "Proteomics TMT label": "ProteomicsTMTLabel",
              "Site of Finding": "SiteOfFinding",
              "Site Subtype 1": "SiteSubtype1",
              "Site Subtype 2": "SiteSubtype2",
              "TMB (nonsynonymous)": "TMBNonSynonymous"              
             }
meta = meta.rename(columns=col_update)

## 2.4 Replace NA's with Unknown

The metadata schema in Tutorial 1 has a set of required fields that must be filled for each sample. NA's are not considered "filled" values. To get around this, we can replace NA's with the string "Unknown", which counts as a valid value in our schema. 

In [65]:
# list of required fields, as stated in metadata schema in Tutorial 1
required = ["Run",
            "SRAStudy",
            "AssayType",
            "AvgSpotLen",
            "BiomaterialProvider",
            "BioProject",
            "BioSample",
            "BioSampleModel",
            "CellLine",
            "CenterName",
            "Consent",
            "Disease",
            "DiseaseStage",
            "Ethnicity",
            "Experiment",
            "Instrument",
            "Isolate",
            "LibraryName",
            "LibraryLayout",
            "LibrarySelection",
            "LibrarySource",
            "Organism",
            "Platform",
            "ReleaseDate",
            "SampleName",
            "SampleType",
            "Sex",
            "Tissue",
            "CreateDate",
            "Version"
    ]

In [66]:
meta[required] = meta[required].fillna(value="Unknown")

## 2.5 Create explicit SampleId field

`Run` is used as the sample ID for studies on SRA. However, this nomenclature is slightly confusing with a sequencing or pipeline run. To circumvent this, we will add an explicit `SampleID` field in the metadata table. 

In [109]:
meta.insert(0, "SampleID", meta["Run"])

# 3. Define CCLE sample subset for demo

Now that we have the sample metadata all cleaned up, we need to define a subset of samples to use in our demo. To best mirror "real world" workflows in the lab, where samples are processed & sequenced on a rolling basis - we decided to select samples that span multiple sequencing/processing batches.


## 3.1 Create sample batch labels

Use date created for batches.

In [67]:
# batch samples based on SRA create date field
batches = meta["CreateDate"].value_counts().to_frame("nSamples")
batches.reset_index(drop=False, inplace = True)
batches.head()

,CreateDate,nSamples
0,2019-02-23T22:52:00Z,16
1,2019-02-23T23:37:00Z,16
2,2019-02-23T22:58:00Z,15
3,2019-02-23T23:01:00Z,15
4,2019-02-24T00:16:00Z,13


In [68]:
# clip off time stamp, so we can use day
# remove last 10 chr ex, T23:37:00Z
# there will be three batches of data based on date w/o time stamp
meta.insert(36, "CreateDateBatch", meta['CreateDate'].map(lambda x: str(x)[:-10]))
meta['CreateDateBatch'].value_counts()

CreateDateBatch
2019-02-23    542
2019-02-24    456
2019-02-25     20
                1
Name: count, dtype: int64

In [69]:
# one sample does not have a batch, drop this sample
meta = meta.loc[meta["CreateDateBatch"] != "", ]

In [70]:
# create a pseudo flow cell ID tag for sample
# to mirror real world processes & approaches to binning NGS samples
meta.insert(1, "FlowCellID", meta["CreateDateBatch"].str.replace("-", "") + "_" + meta["BioProject"] + "_" + meta["Run"])

## 3.2 Subsample samples within each batch

We will only use a subset of samples per batch, for a total of 33 radomly selected samples, for the demo.

1. Batch 2019-02-23 (n=8 samples)
2. Batch 2019-02-24 (n=6 samples)
3. Batch 2019-02-25 (n=19 samples)

In [101]:
# randomly subsample to pre-specified # of samples per batch within each batch

n_samples_per_batch = {"2019-02-23" : 8,
                       "2019-02-24" : 6,
                       "2019-02-25" : 19
                       }

sub_meta = dict()
for batch in n_samples_per_batch:
    print(">>>> " + str(batch))
    df = meta.query("CreateDateBatch == @batch").sample(n_samples_per_batch[batch])
    sub_meta[batch] = df
sub_meta = pd.concat(sub_meta, ignore_index=True)


>>>> 2019-02-23
>>>> 2019-02-24
>>>> 2019-02-25


In [117]:
# save formatted metadata for demo samples to disk
sub_meta.to_csv("./demo_data/sample_metadata/demo_ccle_rnaseq_metadata.csv")

# 4. Download fastqs

After defining our cohort of interest, we can now download raw RNA-sequencing data for the demo samples of interest. Fastqs are downloaded directly from the Sequence Read Archive (SRA) using the [SRA TookKit (v3.0.6)](https://github.com/ncbi/sra-tools/wiki) command `fastq-dump`.

We will use a simple bash script to perform the fastq download: 



```bash

#!/bin/bash

# define location of fastqs to be downloaded to
cd ~/ccle_demo_fastqs/

# for each sample in demo metadata, download fastqs & gzip them
SAMPLES=$(tail -n +2 ./demo_data/sample_metadata/demo_ccle_rnaseq_metadata.csv | awk -F"," '{print $2}')
for SRA in $SAMPLES:
do
    echo $SRA
    prefetch -v $SRA
    fastq-dump --outdir ./fastq/ --split-3 ./$SRA
done;

# gzip fastqs to save space (this is standard practice)
# gzip compression level ranges from 1 to 9; level 1 is fastest, level 6 is default
gzip -3 ~/ccle_demo_fastqs/SRR*

```

Now that we have the raw RNA-sequencing data, we are ready to proceed to the next step - defining a metadata schema and Quilt workflow to ensure the integrity of the metadata when attached to a Quilt package!

----